In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score
import pandas as pd


In [2]:
import pandas as pd
import os

from tqdm import tqdm
    
path = os.getcwd()  # 주피터 노트북 파일 경로
# os_file_list = os.listdir(path)  # 내 경로 읽어서 파일 리스트 읽음

data_frame_samsung = pd.read_excel(path+"\\"+"samsung_laptop2.xlsx")
data_frame_lg = pd.read_excel(path+"\\"+"lg_laptop2.xlsx")
data_frame_apple = pd.read_excel(path +"\\"+"apple_laptop2.xlsx")

combined_df = pd.concat([data_frame_samsung, data_frame_lg, data_frame_apple], ignore_index=True) # 데이터 한번에 결합.

train_combined_df = combined_df[:5120]
test_combined_df = combined_df[5120:]

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
# Load dataset
df = combined_df


In [5]:

# Split into training and testing
train_texts, test_texts, train_labels, test_labels = train_test_split(df['review'], df['label'], test_size=0.2)

# Load pre-trained tokenizer
tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

In [6]:


# Tokenize the texts
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)


In [7]:

class KoreanTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:


# Convert to PyTorch Dataset
train_dataset = KoreanTextDataset(train_encodings, list(train_labels))
test_dataset = KoreanTextDataset(test_encodings, list(test_labels))

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=2)

# Set up training parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [9]:


# # Start training loop
# for epoch in range(5):  # number of epochs
#     loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
#     for batch in loader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs[0]
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

# # Evaluation
# model.eval()

from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Training
model.train()
for epoch in tqdm(range(5), desc='Epochs'):  # number of epochs
    loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    for batch in tqdm(loader, desc='Training'):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()



Training:  35%|███████████████████████▋                                            | 130/373 [44:22<1:18:42, 19.43s/it]


Training:  70%|███████████████████████████████████████████████▊                    | 262/373 [1:26:21<34:55, 18.88s/it]


Training:   5%|███▌                                                                 | 19/373 [05:58<1:51:08, 18.84s/it]


Training:  40%|███████████████████████████▌                                        | 151/373 [47:18<1:09:00, 18.65s/it]


Training:  76%|███████████████████████████████████████████████████▌                | 283/373 [1:28:24<27:43, 18.48s/it]


...

Training:  87%|███████████████████████████████████████████████████████████▏        | 325/373 [1:39:38<14:45, 18.46s/it]


Training:  22%|███████████████▏                                                     | 82/373 [25:14<1:28:54, 18.33s/it]


Training:  57%|███████████████████████████████████████                             | 214/373 [1:07:52<48:42, 18.38s/it]


Training:  93%|███████████████████████████████████████████████████████████████     | 346/373 [1:48:18<08:10, 18.18s/it]


Epochs: 100%|████████████████████████████████████████████████████████████████████████| 5/5 [9:41:21<00:00, 6976.31s/it]


In [10]:
import numpy as np

In [12]:
# Testing/Evaluation
model.eval()
total_eval_accuracy = 0
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
predictions = []
logits_ = []
for batch in tqdm(test_loader, desc='Testing'):
    with torch.no_grad():
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = batch['labels'].to('cpu').numpy()
        total_eval_accuracy += accuracy_score(np.argmax(logits, axis=1), label_ids)
    predictions.extend(predicted)
avg_val_accuracy = total_eval_accuracy / len(test_loader)
print("Accuracy: {0:.2f}".format(avg_val_accuracy))

Testing:   0%|                                                                                  | 0/94 [00:06<?, ?it/s]


NameError: name 'predicted' is not defined

In [ ]:
from tqdm import tqdm
import numpy as np

# Switch model to evaluation mode
model.eval()

# DataLoader for our evaluation set
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Variable to gather all predictions
predictions = []
all_logits = []

# Evaluate data for one epoch
for batch in tqdm(test_loader, desc='Predicting'):
    with torch.no_grad():
        # Forward pass
        outputs = model(**{k: v.to(device) for k, v in batch.items()})
    # Get the logits from the model output
    logits = outputs.logits

    
    # Get the predicted class for each example in the batch (the class with the highest probability)
    _, predicted = torch.max(outputs.logits, 1)
    
    # Move predictions to CPU and convert to NumPy array
    predicted = predicted.cpu().numpy()
    logits = logits.cpu().numpy()
    
    # Append batch predictions
    predictions.extend(predicted)
    all_logits.extend(logits)

# Convert list to a numpy array for further processing or analysis
predictions = np.array(predictions)
all_logits = np.array(all_logits)

In [ ]:
test_labels

In [ ]:
predictions

In [ ]:
torch.save(model, f'./beomi.pt')